In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sqlite3
import regex as re
import matplotlib.pyplot as plt
from wordcloud import WordCloud
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Ham is Called 'not spam' so I changed it to '0' and spam is marked as '1' **

In [ ]:
#create dataframe from csv
df = pd.read_csv('/kaggle/input/spam-text-message-classification/SPAM text message 20170820 - Data.csv',encoding='ISO-8859-1')
df["Category"].replace({'ham': '0','spam': '1'}, inplace=True)
df.head()

In [ ]:
df['Category'] = df['Category'].astype(int)

In [ ]:
df.info()

# Exploratory Analysis

Before anything, it is best to do a quick analysis of the data to eliminate duplicate rows and establish some baseline counts. I use pandas drop_duplicates to drop the duplicate rows.

In [ ]:
print("spam count: " +str(len(df.loc[df.Category==1])))
print("not spam count: " +str(len(df.loc[df.Category==0])))
print(df.shape)

df = df.drop_duplicates()
df = df.reset_index(inplace = False)[['Category','Message']]
print(df.shape)

#  Word Cloud

**Word clouds are a useful way to visualize text data because they make understanding word frequencies easier. Words that appear more frequently within the email text appear larger in the cloud. Word Clouds make it easy to identify “key words.”**

In [ ]:
clean_desc = []
for w in range(len(df.Message)):
    desc = df['Message'][w].lower()
    
    #remove punctuation
    desc = re.sub('[^a-zA-Z]', ' ', desc)
    
    #remove tags
    desc=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",desc)
    
    #remove digits and special chars
    desc=re.sub("(\\d|\\W)+"," ",desc)
    
    clean_desc.append(desc)
#assign the cleaned descriptions to the data frame
df['Message'] = clean_desc
        
df.head(3)

In [ ]:
stop_words = ['is','you','your','and', 'the', 'to', 'from', 'or', 'I', 'for', 'do', 'get', 'not', 'here', 'in', 'im', 'have', 'on', 're', 'new', 'subject']

In [ ]:
wordcloud = WordCloud(width = 800, height = 800, background_color = 'black', stopwords = stop_words, max_words = 1000, min_font_size = 20).generate(str(df['Message']))
#plot the word cloud
fig = plt.figure(figsize = (8,8), facecolor = None)
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

# Spam Detection

**This is a binary classification problem since an email can either be spam (1) or not spam (0). I want to build a machine learneing model that can identify whether or not an email is spam. I am going to use the Python library Scikit-Learn to explore tokenization, vectorization, and statistical classification algorithms.**

**Import Dependencies**

Import the Scikit-Learn functionality we need to transform and model the data. I will use CountVectorizer, train_test_split, ensemble models, and a couple metrics.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn import ensemble 
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
#list of sentences
text = df.Message
#instantiate the class
cv = CountVectorizer()
#tokenize and build vocab
cv.fit(text)
print(cv.vocabulary_)
#transform the text
vector = cv.transform(text)
print(vector.toarray())

**The CountVectorizer is counting the tokens and allowing me to construct the sparse matrix containing the transformed words to numbers.**

Bag of Words Method

In [ ]:
text_vec = CountVectorizer().fit_transform(df['Message'])
X_train, X_test, y_train, y_test = train_test_split(text_vec, df['Category'], test_size = 0.90, random_state = 75, shuffle = True)

**The Classifier**

**I am using the GradientBoostingClassifier() model from the Scikit-Learn Ensemble collection.**

In [ ]:
classifier = ensemble.GradientBoostingClassifier(
    n_estimators = 500, #how many decision trees to build
    learning_rate = 2.0, #learning rate
    max_depth = 500
)

**Generate Predictions**

Finally, we fit the data, call predict and generate the classification report. Using classification_report(), it is easy to build a text report showing the main classification metrics.

In [ ]:
classifier.fit(X_train, y_train)
predictions = classifier.predict(X_test)
print(classification_report(y_test, predictions))

**our model achieved 95% accuracy**